# Journal Transcript Analysis

Journal entry is converted from speech-to-text and then analysed:
- Short summary of entry
- Tags extracted for topics discussed
- Key emotions detected

A local file, **api_keys.py**, is required in the same directory as the notebook which contains variables with the api keys associated with the different services. You will need to create accounts with [AssemblyAI](https://www.assemblyai.com) and [OpenAI](https://openai.com) to get the api keys.

Current **api_keys.py** file contains:

aai_api_key = {YOUR_ASSEMBLYAI_API_KEY}

openai_api_key = {YOUR_OPENAI_API_KEY}

You will also need install these packages:
- assemblyai
- langchain_community

In [1]:
import assemblyai as aai
from langchain_community.document_loaders import AssemblyAIAudioTranscriptLoader
from api_keys import aai_api_key, openai_api_key

In [2]:
# test journal stored under the filename "test_journal.m4a"
audio_file = "test_journal.m4a"

# configuration for the model performing the transcription
config = aai.TranscriptionConfig(
    sentiment_analysis = True,
    summarization=True
)

In [4]:
loader = AssemblyAIAudioTranscriptLoader(file_path=audio_file,
                                         api_key=aai_api_key,
                                         config=config)
docs = loader.load()

In [5]:
docs[0].page_content
# docs[0].metadata['audio_duration']
# docs[0].metadata['confidence']
# docs[0].metadata['words'][0]
# docs[0].metadata['summary']
# docs[0].metadata['sentiment_analysis_results'][7]

"You. My parents used to say college is a new world. It's a new experience. At this moment I can say I am satisfied and proud to have taken the decision to study. College opened doors to many opportunities and adventures that I still have yet to live and discover. I'm excited to meet each and one of my professors and know what challenges each of my classes will bring. I know for now that it will be an unforgettable experience. However, knowing how anxious I am to experience each of these adventures, I think I still have things to learn, fix and understand. I'm a very shy person and I hate talking in front of people. Also, I have a hard time keeping up with professors and understanding what I read. There are many things I have to fix but that doesn't limit me to try and give my best. I know that with God's help and my family I accomplish. I will accomplish this and more."

In [5]:
pos, neu, neg = 0, 0, 0

# docs[0].metadata['sentiment_analysis']
for i in docs[0].metadata['sentiment_analysis_results']:
    sentiment = i['sentiment'].value
    if sentiment == "POSITIVE":
        pos += 1
    elif sentiment == "NEUTRAL":
        neu += 1
    elif sentiment == "NEGATIVE":
        neg += 1

print(pos, neu, neg)

7 3 3


---
Using AssemblyAI API directly, rather than through LangChain

In [6]:
aai.settings.api_key = aai_api_key

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(audio_file, config)

In [13]:
# transcript.text
# transcript.words
# transcript.sentiment_analysis
# transcript.summary

In [ ]:
# use lemur llm from AssemblyAI to summarise the transcript into emotions - have to pay to use api
result = transcript.lemur.task(
    prompt="Summarise the feelings in this transcript into a few key words",
    temperature=0.5
)

In [ ]:
# possible to remove all data relating to request - good for controlling data
aai.Lemur.purge_request_data(result.request_id)

---
Using OpenAI API


In [12]:
from openai import OpenAI

In [ ]:
client = OpenAI(api_key=openai_api_key)

In [30]:
journal = open(audio_file, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=journal, 
  response_format="text"
)

In [15]:
print(transcription)

My parents used to say, college is a new world, it's a new experience. At this moment I can say I am satisfied and proud to have taken the decision to study. College opened doors to many opportunities and adventures that I still have yet to live and discover. I'm excited to meet each and one of my professors and know what challenges each of my classes will bring. I know for now that it will be an unforgettable experience. However, knowing how anxious I am to experience each of these adventures, I think I still have things to learn, fix and understand. I'm a very shy person and I hate talking in front of people. Also, I have a hard time keeping up with professors and understanding what I read. There are many things I have to fix, but that doesn't limit me to try and give my best. I know that with God's help and my family I will accomplish this and more.



In [9]:
prompt=f"Summarise the following journal transcript into a few sentences"

llm.invoke(
    input = f"{prompt} : {docs[0].page_content}"
)

'\n\nThe author expresses their satisfaction and pride in their decision to study in college, as it has opened doors to new opportunities and adventures. They are excited to meet their professors and face the challenges of their classes, but also acknowledge their shyness and struggles with public speaking and understanding material. However, they are determined to overcome these obstacles with the help of God and their family.'

In [16]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant, analysing journal entries to help users understand \
    feelings they might be experiencing."},
    {"role": "user", "content": f"{prompt} : {transcription}"} # docs[0].page_content
  ]
)

In [29]:
response.choices[0].message.content

'The writer reflects on the excitement and pride of entering college, seeing it as a new world full of opportunities and challenges. They express eagerness to meet professors and face the adventures ahead, but also acknowledge feelings of anxiety, shyness, and struggles with public speaking and academics. Despite these challenges, they are determined to overcome them with the support of their family and faith in their abilities.'

In [39]:
prompt=f"Summarise the feelings in the following transcript into the three key emotions. The output should be a list of \
    single emotions only."

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant, analysing journal entries to help users understand \
    feelings they might be experiencing."},
    {"role": "user", "content": f"{prompt} : {transcription}"} # docs[0].page_content
  ]
)

In [40]:
response.choices[0].message.content

'- Satisfaction\n- Excitement\n- Anxious'

In [43]:
prompt=f"Extract three tags from the following transcript which are not emotions. The output should be a list of single keywords \
    regarding what the user is talking about. For example, if the user was talking about a time when they went skiing, holiday \
        and skiing might be two of the keywords returned."

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant, analysing journal entries to help users understand \
    feelings they might be experiencing."},
    {"role": "user", "content": f"{prompt} : {transcription}"} # docs[0].page_content
  ]
)

In [44]:
response.choices[0].message.content

'1. College\n2. Challenges\n3. Opportunities'

---